# CHEQUEOS POST LIQUIDACION

### Obtencion de Datos

In [96]:
import pyodbc as mssql
import numpy as np
import pandas as pd
%config IPCompleter.greedy=True
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from IPython.display import Markdown as md

In [97]:
liquidacion = '2022-04'
minimo_general = 14102.10
minimo_policias = 44538.11
promedio_incrementos = 10.00/100.00
excluir_reparticiones_minimo_garantizado = [17, 26]

altas_desde = ''
#      no estaban incorporados a 04-2022                                                v    v    v    v     v    
codigos_bruto = ['0Q','0R','0N','0S','LC','OC','PC','QC','TC','U0','MC','PA','TE','XX','PX','MX','YX','NX','1X']
# codigos_bruto = ['0Q','0R','0N','0S','LC','OC','PC','QC','TC','U0','MC','PA','TE']

In [98]:
conn = mssql.connect("DRIVER={SQL SERVER};SERVER=IPSBD;DATABASE=LIQUIDACION;;Trusted_Connection=Yes;")

#### Obtención de la fecha de alta de la liquidacion seleccionada

In [99]:
from datetime import datetime
altas_desde = pd.read_sql("""SELECT create_date FROM SYS.DATABASES
WHERE NAME like '%{liquidacion}%'""".format(liquidacion=liquidacion),conn)["create_date"][0].strftime('%d/%m/%Y')
print("ALTAS DESDE: {altas_desde}".format(altas_desde=altas_desde))

ALTAS DESDE: 19/04/2022


#### Obtención de Datos Generales

In [100]:
codigos = pd.read_sql( "SELECT * FROM CODIGOS", conn, index_col="CODIGO")
reparticion = pd.read_sql( "SELECT * FROM REPARTICION", conn, index_col="Cod" )
tipos_beneficio = pd.read_sql ( "SELECT * FROM BENEFICIO", conn, index_col="Cod" )

auditorias = pd.read_sql( """SELECT CONTROL FROM AUDIT A WHERE A.TIPO IN ('A', 'M') AND
A.ORIGEN='D' AND A.FECHA > CONVERT(int, CONVERT(datetime, '{altas_desde}'))-2""".format(altas_desde=altas_desde), conn)

beneficios = pd.read_sql( """SELECT CONTROL, PLLA, ORDEN, AFILIADO, BENEFICIO, COD_BENEFICIO, LETRA,
CASE WHEN COD_BENEFICIO in (7,25,27,29) then DBO.PORCENTAJE_COMPARTIDA(CONTROL) ELSE 1 END AS PORCENTAJE
FROM Liquidacion.dbo.DatosAfiliado
WHERE HABILITADO='S'""", conn, index_col="CONTROL")
beneficios[["COD_BENEFICIO"]]=beneficios[["COD_BENEFICIO"]].astype("int64")
### Excluímos las auditorias de altas y modificaciones para evitar las "deformaciones"
beneficios = beneficios[~beneficios.isin(auditorias["CONTROL"])]
liquidacion_ben = pd.read_sql(""" SELECT * FROM LIQUIDACION.DBO.LIQUIDACION""", conn, index_col=["PLLA", "ORDEN", "AFILIADO"])
beneficios_liq = pd.read_sql("SELECT * FROM [{liquidacion}].DBO.DATOSAFILIADO".format(liquidacion=liquidacion), \
                             conn, index_col=["CONTROL"])
liquidacion_liq = pd.read_sql("SELECT * FROM [{liquidacion}].DBO.LIQUIDACION".format(liquidacion=liquidacion), \
                                conn, index_col=["PLLA", "ORDEN", "AFILIADO"])


### Determinacion de brutos

#### Determinacion de los brutos actuales

In [101]:
### determinacion de la tabla de brutos de liquidacion activa
brutos_act = beneficios.reset_index().merge(liquidacion_ben, left_on=["PLLA", "ORDEN", "AFILIADO"],\
                              right_on=["PLLA", "ORDEN", "AFILIADO"])
brutos_act = brutos_act[brutos_act["CODIGO"].isin(codigos_bruto)].groupby(["CONTROL"]).sum("MONTO")[["MONTO"]]
# agrego aquellos que no tienen codigos de bruto
brutos_act = brutos_act.join(beneficios,  how="outer")
brutos_act["MONTO"].fillna(value=0.0, inplace=True)
brutos_act["BRUTO"] = brutos_act["MONTO"] + brutos_act["BENEFICIO"]
# dejo solo la columna del bruto recien calculada
brutos_act=brutos_act[["BRUTO","BENEFICIO", "COD_BENEFICIO", "LETRA"]]



#### Determinación de los brutos de referencia

In [102]:
###determinacion de la tabla de brutos de la liquidacion de referencia 
brutos_liq = beneficios_liq.join(liquidacion_liq, on=["PLLA","ORDEN","AFILIADO"],how="inner", rsuffix=".LIQ")\
        .join(codigos, on=["CODIGO"], how="inner", rsuffix=".COD")
brutos_liq = brutos_liq[brutos_liq["CODIGO"].isin(codigos_bruto)].groupby(by="CONTROL").sum()[["MONTO"]]
brutos_liq = brutos_liq.join(beneficios_liq, on="CONTROL", how="outer")
brutos_liq["MONTO"].fillna(value=0.0, inplace=True)
brutos_liq["BRUTO"] = brutos_liq["MONTO"] + brutos_liq["BENEFICIO"]
brutos_liq=brutos_liq[["CONTROL", "BRUTO", "BENEFICIO"]]
brutos_liq.set_index(keys="CONTROL", inplace=True)
brutos_liq=brutos_liq.sort_values(by="CONTROL")


### CALCULO Y COMPARATIVA DE LOS BRUTOS Y BENEFICIOS

In [103]:
###comparativa 

brutos = brutos_liq.join( brutos_act, lsuffix="_liq", rsuffix="_act" )
brutos["DIF_BRUTOS"] = (brutos["BRUTO_act"]-brutos["BRUTO_liq"])/brutos["BRUTO_liq"]
brutos["DIF_BENEFICIOS"] = (brutos["BENEFICIO_act"]-brutos["BENEFICIO_liq"])/brutos["BENEFICIO_liq"]
diferencia_brutos=brutos.loc[(brutos["DIF_BRUTOS"] > promedio_incrementos) | (brutos["DIF_BENEFICIOS"] > promedio_incrementos), ["DIF_BRUTOS","DIF_BENEFICIOS"]].sort_values(by="DIF_BRUTOS", ascending=False)

diferencia_brutos[~brutos.isin(auditorias["CONTROL"]) ]


,DIF_BRUTOS,DIF_BENEFICIOS
CONTROL,,
36358,3.86,3.86
38235,3.63,3.84
72259,2.65,1.81
519,2.63,2.63
61584,2.20,1.81
...,...,...
57795,-0.09,0.14
72099,-0.42,0.10
69251,-0.66,0.10


In [104]:
### POR REPARTICION ###

inf_rep_letra = brutos.merge(reparticion, left_on="LETRA", right_on="Cod", how="inner")
inf_rep_letra = inf_rep_letra[['Detalle', 'BRUTO_liq', 'BENEFICIO_liq', 'BRUTO_act', 'BENEFICIO_act']].groupby(by="Detalle").sum()
inf_rep_letra["DIF_BRUTOS"] = (inf_rep_letra["BRUTO_act"]-inf_rep_letra["BRUTO_liq"])/inf_rep_letra["BRUTO_liq"]
inf_rep_letra["DIF_BENEFICIOS"] = (inf_rep_letra["BENEFICIO_act"] - inf_rep_letra["BENEFICIO_liq"])/inf_rep_letra["BENEFICIO_liq"]

inf_rep_letra.to_excel("comparativa.xlsx")


### Mínimos

#### Mínimos Administración Central y Organismos

In [105]:
minimos = beneficios.join( beneficios_liq, on="CONTROL", lsuffix=".BEN", rsuffix=".BENLIQ" )\
            .join(reparticion, on="LETRA" , rsuffix=".REP", )\
            .join(brutos, on="CONTROL", rsuffix=".BRT")

minimos[minimos["BRUTO_act"] < minimo_general]
minimos["ZX_act"] = np.where((~minimos["LETRA"].isin([17,26]) & (minimos["BRUTO_act"] < minimo_general*minimos["PORCENTAJE"])), minimo_general*minimos["PORCENTAJE"]-minimos["BRUTO_act"], 0.0)
minimos.fillna(0, inplace=True)

minimos_pagados = beneficios_liq.join(liquidacion_liq, on=["PLLA","ORDEN","AFILIADO"],how="inner", rsuffix=".LIQ")\
        .join(codigos, on=["CODIGO"], how="inner", rsuffix=".COD")

minimos_pagados=minimos_pagados.loc[minimos_pagados["CODIGO"].isin(['ZX']), ["MONTO"]].groupby("CONTROL").sum("MONTO").rename(columns={"MONTO":"ZX_liq"})

minimos=minimos.join(minimos_pagados, on="CONTROL", rsuffix=".MINPAG")
minimos.fillna(0.0, inplace=True)
minimos=minimos.loc[(np.abs(minimos["ZX_act"]-minimos["ZX_liq"])>1)  ,["BRUTO_act", "ZX_act", "BRUTO_liq", "ZX_liq"]]

minimos.to_excel("Minimos_Adm_Central" + liquidacion + ".xlsx")



Mínimos Retiros Policiales y Carcelarios

In [106]:
minimos = beneficios.join( beneficios_liq, on="CONTROL", lsuffix=".BEN", rsuffix=".BENLIQ" )\
            .join(reparticion, on="LETRA" , rsuffix=".REP", )\
            .join(brutos, on="CONTROL", rsuffix=".BRT")\
            .join(tipos_beneficio, on="COD_BENEFICIO", rsuffix=".tb")
            

minimos[minimos["BRUTO_act"] < minimo_policias]
minimos["PX_act"] = np.where((minimos["LETRA"].isin([17,26]) & (minimos["BRUTO_act"] < minimo_policias*minimos["PORCENTAJE"])), minimo_policias*minimos["PORCENTAJE"]-minimos["BRUTO_act"], 0.0)
minimos.fillna(0, inplace=True)

minimos_pagados = beneficios_liq.join(liquidacion_liq, on=["PLLA","ORDEN","AFILIADO"],how="inner", rsuffix=".LIQ")\
        .join(codigos, on=["CODIGO"], how="inner", rsuffix=".COD")

minimos_pagados=minimos_pagados.loc[minimos_pagados["CODIGO"].isin(['PX']), ["MONTO"]]\
        .groupby("CONTROL").sum("MONTO")

minimos=minimos.join(minimos_pagados, on="CONTROL", rsuffix=".MINPAG").rename(columns={"MONTO":"PX_liq", "Detalle":"Reparticion", "Detalle.tb":"TIPO_BEN"})
minimos.fillna(0.0, inplace=True)

minimos=minimos.loc[(np.abs(minimos["PX_act"]-minimos["PX_liq"])>1)  ,["Reparticion", "TIPO_BEN", "BRUTO_act", "PX_act", "BRUTO_liq", "PX_liq"]]
minimos.to_excel("Minimos_RetirosPoliciales" + liquidacion + ".xlsx")

minimos

,Reparticion,TIPO_BEN,BRUTO_act,PX_act,BRUTO_liq,PX_liq
CONTROL,,,,,,
1479,Retiro Policial,Jubilación por Retiro Voluntario,49048.51,0.00,44538.11,360.42
3236,Retiro Policial,Jubilación Ordinaria,49048.51,0.00,44538.11,2535.06
3414,Retiro Policial,Jubilación por Invalidez,49048.51,0.00,44538.11,2535.06
3418,Retiro Policial,Jubilación por Retiro anticipado,49048.51,0.00,44538.11,2535.06
3422,Retiro Policial,Jubilación por Invalidez,49048.51,0.00,44538.11,2535.06
...,...,...,...,...,...,...
73323,Retiro Policial,Pension Derivada,39530.41,5007.70,35578.09,0.00
73482,Retiro Policial,Menores Pensionados,41556.41,2981.70,37738.87,0.00
73493,Retiro Policial,Pension Automatica,42221.07,2317.04,38342.96,0.00
